# RAG Structured Output
LLM JSON Schema를 원하는 구조로 응답하도록 처리할 수 있다.

`llm.with_structured_output(PydanticModelClass)`

In [1]:
%pip install langchain langchain-openai -Uq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")

In [3]:
from langchain_core.documents import Document

def retrieve_vectordb(query=None):
    return [
        Document(page_content="파리는 프랑스의 수도로, 연간 관광객 수 약 3000만 명입니다. 주요 관광지로는 센강, 개선문, 루브르 박물관이 있습니다."),
        Document(page_content="런던은 영국의 수도로, 연간 관광객 수 약 2000만 명입니다. 주요 관광지로는 버킹엄 궁전, 런던 아이, 타워 브릿지가 있습니다."),
        Document(page_content="교토는 일본의 옛 수도로, 연간 관광객 수 약 1500만 명입니다. 주요 관광지로는 금각사, 은각사, 기요미즈데라가 있습니다.")
    ]

retrieve_vectordb()

[Document(metadata={}, page_content='파리는 프랑스의 수도로, 연간 관광객 수 약 3000만 명입니다. 주요 관광지로는 센강, 개선문, 루브르 박물관이 있습니다.'),
 Document(metadata={}, page_content='런던은 영국의 수도로, 연간 관광객 수 약 2000만 명입니다. 주요 관광지로는 버킹엄 궁전, 런던 아이, 타워 브릿지가 있습니다.'),
 Document(metadata={}, page_content='교토는 일본의 옛 수도로, 연간 관광객 수 약 1500만 명입니다. 주요 관광지로는 금각사, 은각사, 기요미즈데라가 있습니다.')]

## RAG 체인 생성

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field  # 출력 스키마(검증/구조화) 정의용
from typing import List  # 리스트 타입 힌트

# 출력 schema 클래스 정의
class CityInfo(BaseModel):
    '''도시 관광 정보'''
    city: str = Field(description='도시 이름')
    visitors: int = Field(description='연간 방문객수')  
    landmarks: List[str] = Field(description='주요 관광지 목록')

# 여러 도시 정보를 한 번에 담는 응답 스키마
class CityList(BaseModel):
    '''도시 정보 목록'''
    cities: List[CityInfo]

llm = init_chat_model('gpt-4.1-mini')
prompt = PromptTemplate.from_template('''
당신은 조회된 문서에서 사용자 원하는 정보를 추출하는 에이젼트입니다.
제공된 문서를 바탕으로 JSON형식으로 답변해주세요.

[조회된 문서]
{docs}

[사용자 질문]
{query}
''')

chain = prompt | llm.with_structured_output(CityList)  # 응답을 CityList 스키마로 강제

context = '\n\n'.join([doc.page_content for doc in retrieve_vectordb()])  # 문서 본문을 하나로 결합
query = '교토의 주요 관광 정보는?'
response: CityList = chain.invoke({'docs': context, 'query': query})  # 문서 + 질의로 구조화된 응답 생성
print(response)  # Pydantic 객체 형태로 출력

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cities=[CityInfo(city='교토', visitors=15000000, landmarks=['금각사', '은각사', '기요미즈데라'])]


In [5]:
query = '교토, 파리의 주요 관광 정보는?'
response: CityList = chain.invoke({'docs': context, 'query': query})
print(response)

cities=[CityInfo(city='교토', visitors=15000000, landmarks=['금각사', '은각사', '기요미즈데라']), CityInfo(city='파리', visitors=30000000, landmarks=['센강', '개선문', '루브르 박물관'])]


In [ ]:
query = '서울의 주요 관광 정보는?'
response: CityList = chain.invoke({'docs': context, 'query': query})
print(type(response))  # 응답 객체 타입은 CityList
print(response)

<class '__main__.CityList'>
cities=[]


## Pydantic 문법

In [7]:
from pydantic import BaseModel, Field
from typing import List, Dict, Annotated
from datetime import datetime

class User(BaseModel):
    '''
    pydantic BaseModel클래스에서는 멤버변수를 아래와 같이 선언한다.
    '''
    id: int # 필수값
    name: str
    email: str = None # 옵션
    is_active: bool = True # 옵션 (기본값: True)
    created_at: datetime

honggd = User(
    id=1,
    name='홍길동',
    email='honggd@naver.com',
    is_active=True,
    created_at =datetime.now()
)
print(honggd) # 속성 내용 확인

sinsa = User(
    id="2", # 자동타입변환
    name='신사임당',
    created_at = datetime.now()
)
print(sinsa)

id=1 name='홍길동' email='honggd@naver.com' is_active=True created_at=datetime.datetime(2026, 2, 9, 15, 34, 15, 532836)
id=2 name='신사임당' email=None is_active=True created_at=datetime.datetime(2026, 2, 9, 15, 34, 15, 534350)


In [ ]:
# 유효성 검사
from pydantic import ValidationError  # Pydantic 검증 예외 타입

try:
    leess = User(
        id="leess",
        name='이순신',
        created_at = datetime.now()
    )
    print(leess)
except ValidationError as e:
    print('User 검증오류:\n', str(e))

User 검증오류:
 1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='leess', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [ ]:
# Field를 사용한 메타정보/검증정보 작성
class Product(BaseModel):
    code: str = Field(..., description='제품식별코드') # ... 필수값 선언
    name: str = Field(..., description='제품명', min_length=1, max_length=10)  # 길이 제한(1~10)
    price: float = Field(..., description='제품가격', ge=0)   # 0 이상만 허용
    quantity: int = Field(default=0, description='제품수량')  # 기본값 0
    description: str = Field(..., description='제품설명', max_length=500)  # 최대 500자
    discount_rate: float = Field(default=0.0, description='제품할인율', ge=0, le=1)  # 0~1 범위 제한
    created_at: datetime = Field(default_factory=datetime.now, description='제품등록일')  # 생성 시간 자동 채움

In [10]:
prod = Product(
    code='prd_123',
    name='텀블러',
    price=30_000,
    discount_rate=0.15,
    description='멋진 텀블러')
print(prod)

code='prd_123' name='텀블러' price=30000.0 quantity=0 description='멋진 텀블러' discount_rate=0.15 created_at=datetime.datetime(2026, 2, 9, 15, 34, 15, 591714)


In [ ]:
# json으로 변환
prod_json_str = prod.model_dump_json()  # Pydantic 객체를 JSON 문자열로 직렬화
print(prod_json_str)

# dict로 변환
prod_dict = prod.model_dump()  # Pydantic 객체를 dict로 변환
print(prod_dict)

# json에서 pydantic클래스 변환
json_str = '{"code":"prd_123","name":"텀블러","price":30000.0,"quantity":0,"description":"멋진 텀블러","discount_rate":0.15,"created_at":"2026-01-21T01:11:24.128594"}'
prod_from_json = Product.model_validate_json(json_str)  # JSON -> Product로 파싱/검증
print(type(prod_from_json), prod_from_json)

# dict에서 pydantic클래스 변환
data = {'code': 'prd_123', 'name': '텀블러', 'price': 30000.0, 'quantity': 0, 'description': '멋진 텀블러', 'discount_rate': 0.15, 'created_at': datetime(2026, 1, 21, 1, 11, 24, 128594)}
prod_from_dict = Product(**data)  # dict를 언패킹으로 Product 생성 (검증 포함)
# prod_from_dict = Product.model_validate(data)  # 다른 방식
print(type(prod_from_dict), prod_from_dict)

{"code":"prd_123","name":"텀블러","price":30000.0,"quantity":0,"description":"멋진 텀블러","discount_rate":0.15,"created_at":"2026-02-09T15:34:15.591714"}
{'code': 'prd_123', 'name': '텀블러', 'price': 30000.0, 'quantity': 0, 'description': '멋진 텀블러', 'discount_rate': 0.15, 'created_at': datetime.datetime(2026, 2, 9, 15, 34, 15, 591714)}
<class '__main__.Product'> code='prd_123' name='텀블러' price=30000.0 quantity=0 description='멋진 텀블러' discount_rate=0.15 created_at=datetime.datetime(2026, 1, 21, 1, 11, 24, 128594)
<class '__main__.Product'> code='prd_123' name='텀블러' price=30000.0 quantity=0 description='멋진 텀블러' discount_rate=0.15 created_at=datetime.datetime(2026, 1, 21, 1, 11, 24, 128594)


## 실습 정리 (RAG + Structured Output)

- 본 실습에서는 RAG 파이프라인에서 LLM의 응답을 **Structured Output(JSON)** 형태로 받는 방법을 다루었다.
- 조회된 문서를 기반으로 답변을 생성하되, `Pydantic(BaseModel)` 스키마를 통해  
  **응답 형식과 필드 구조를 강제**함으로써 결과의 일관성과 신뢰성을 확보했다.
- Structured Output을 적용하면  
  - 필요한 정보만 정확히 추출  
  - 불필요한 자연어 응답 최소화  
  - 후처리 코드 단순화  
  가 가능하다.
- 이는 RAG를 **검색 → 생성 → 응답 표시** 수준이 아닌,  
  **검색 → 정보 추출 → 시스템 연계** 단계로 확장할 수 있게 해준다.
- 실무에서는 Structured Output 기반 RAG가  
  - 사내 문서 정보 추출  
  - 고객 응답 데이터 정형화  
  - API/DB 연동 자동화  
  등에 활용되며, 운영 가능한 서비스 형태의 RAG 구현에 필수적인 패턴이다.